In [ ]:
# !pip install langchain_community
# !pip install langchain_openai
# !pip install llama-index-llms-langchain
# !pip install python-dotenv

In [ ]:
# !pip install llama_index
# !pip install llama-index-embeddings-huggingface
# !pip install llama-index-llms-huggingface

from llama_index.core import SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex

In [16]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
load_dotenv("api_key.env")

True

In [ ]:
loader = SimpleDirectoryReader(
    input_dir=".",
    recursive=True,
    required_exts=[".pdf"],
)

# Load the documents
documents = loader.load_data()
documents

In [20]:
embedding_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

print(embedding_model._model.device)  # Device that the model is running on

cpu


In [22]:
# Creates embeddings for the sentences and stores them
index = VectorStoreIndex.from_documents(
    documents,
    embed_model=embedding_model,
)

# Save the index in the current directory
index.storage_context.persist(persist_dir="./huggingfaceembeddings")


In [23]:
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embedding_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
storage_context = StorageContext.from_defaults(persist_dir="./huggingfaceembeddings")
index = load_index_from_storage(storage_context, embed_model=embedding_model)

In [24]:
# Viewing the chunks
for doc in index.docstore.docs.values():
    print("Document ID:", doc.ref_doc_id)
    print("Text Chunk:", doc.text)
    print("=" * 50)


Document ID: 35948430-ebd2-40f7-94ee-00d761265658
Text Chunk: Shraddha Piparia Computational Biologist, Richland, WA | +1-940-297-9424 | spiparia@health.ucsd.edu Professional Experience Postdoctoral Research Associate | 2021-Present | University of California, San Diego  • Conducted research on asthma pharmacogenetics and computational epidemiology, integrating genomic sequencing data with clinical phenotypes • Developed statistical models and machine learning pipelines to identify genetic determinants in respiratory diseases • Collaborated with multidisciplinary teams in a remote setting, contributing to high-impact publications Application Developer | 2013-2016 | Oracle India Private Limited | Telangana, India • Developed enterprise-level applications and ML-based sentiment analysis tools • Enhanced software testing processes through automated test case generation Technical Skills Programming: R, Python, C, SQL, MySQL, PostgreSQL, git/GitHub, docker, AWS ML Framework: scikit-learn, s

In [25]:
os.environ["OPENAI_API_KEY"] = os.getenv("api_key")

openai_llm = ChatOpenAI(temperature=0.7, model_name="gpt-4o-mini")

query_engine = index.as_query_engine(llm=openai_llm)

In [27]:
while True:
    question = input("Question: ")
    if question.lower() == "quit":
        break
    print(query_engine.query(question).response)


Question: what is the document about?
The document is a resume belonging to Shraddha Piparia, a computational biologist based in Richland, WA. It outlines her educational background, including a PhD in Computer Science and other degrees, as well as her professional experience in research and application development. The resume highlights her current role as a postdoctoral research associate focused on asthma pharmacogenetics and computational epidemiology, her previous experience at Oracle India, and her technical skills in programming, machine learning, bioinformatics, and statistical analysis. Additionally, it lists selected publications related to her research work.
Question: summarize
Shraddha Piparia is a computational biologist based in Richland, WA, currently serving as a Postdoctoral Research Associate at the University of California, San Diego since 2021. She holds a PhD in Computer Science from the University of North Texas, an MTech in Software Engineering from NIT Rourkela,